<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.851 · Tipología y ciclo de vida de los datos · Práctica 1</p>
<p style="margin: 0; text-align:right;">2021-2 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

******
   # Práctica 1: Web scraping
******

<div class="alert alert-block alert-info">
<strong>Autores: CARLOS MAS ESTÉVEZ Y CLAUDIA REINA FAJARDO</strong>
</div>

In [177]:
# Importamos las librerías necesarias
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

En primer lugar, haremos una prueba con la página principal de la web escogida, que nos ofrece la tarifa del presente día.

A través de esta toma de contacto, crearemos un dataset con tantas columnas como la fecha y el horario y con una sola fila con todos los precios del día.

In [178]:
# Fijamos url de la web
url = 'https://tarifaluzhora.es/'

# Hacemos la petición a la página
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [179]:
# Obtenemos las horas
horas = soup.find_all('span', itemprop="description")

# Obtenemos los precios
precios = soup.find_all('span', itemprop="price")

# Obtenemos la fecha
date = soup.find('input', {'name': 'date'}).get('value')

# Creamos un array con el contenido de las horas
columnas = ['fecha']
for h in horas:
    columnas.append(h.text)
    
# Creamos un array con el contenido de los precios
contenido = [date]
for p in precios:
    contenido.append(p.text)

In [180]:
# Creamos un dataset con los datos del primer día, cuyas columnas son la fecha y las horas
df = pd.DataFrame(data=[np.array(contenido)], columns=columnas)
df

,fecha,00h - 01h:,01h - 02h:,02h - 03h:,03h - 04h:,04h - 05h:,05h - 06h:,06h - 07h:,07h - 08h:,08h - 09h:,...,14h - 15h:,15h - 16h:,16h - 17h:,17h - 18h:,18h - 19h:,19h - 20h:,20h - 21h:,21h - 22h:,22h - 23h:,23h - 24h:
0,30/03/2022,0.33468 €/kWh,0.31902 €/kWh,0.30903 €/kWh,0.29593 €/kWh,0.29989 €/kWh,0.3092 €/kWh,0.32472 €/kWh,0.32824 €/kWh,0.37147 €/kWh,...,0.33943 €/kWh,0.33612 €/kWh,0.32761 €/kWh,0.31185 €/kWh,0.39006 €/kWh,0.4168 €/kWh,0.43635 €/kWh,0.44541 €/kWh,0.35615 €/kWh,0.35182 €/kWh


Cuando, mediante el formulario en forma de calendario de la web accedemos a una fecha anterior, se nos redirige a una url con el siguiente formato:
* https://tarifaluzhora.es/?tarifa=pcb&fecha={DD}%2F{MM}%2F{YYYY}
Donde:
* {DD} es el día con dos dígitos
* {MM} es el mes con dos dígitos
* {YYYY} es el año con cuatro dígitos
También hemos observado que no ofrece datos más allá del 01 de junio de 2021.

Con toda esta información procedemos a crear un array de urls que recorreremos para realizar la extracción y crear la base de datos.

In [181]:
# Creamos array vacio de urls
urls=[]

# Recorremos rango de fechas hacia atrás
for i in range(2022,2020,-1):
    # Si la fecha es 2022 solo queremos los tres primeros meses
    if i==2022:
        for j in range(3,0,-1):
            # Para febrero sólo recorremos 28 días
            if j==2:
                for k in range(28,0,-1):
                    url = 'https://tarifaluzhora.es/?tarifa=pcb&fecha='+str(k).zfill(2)+'%2F'+str(j).zfill(2)+'%2F'+str(i)
                    urls.append(url)
            else:
                # Para el resto de meses recorremos 31
                for k in range(31,0,-1):
                    url = 'https://tarifaluzhora.es/?tarifa=pcb&fecha='+str(k).zfill(2)+'%2F'+str(j).zfill(2)+'%2F'+str(i)
                    urls.append(url)
    # Si la fecha es 2021 solo queremos hasta junio
    else:
        for j in range(12,5,-1):
            # Para junio, septiembre y noviembre recorremos 30 días
            if (j==6) | (j==9) | (j==11):
                for k in range(30,0,-1):
                    url = 'https://tarifaluzhora.es/?tarifa=pcb&fecha='+str(k).zfill(2)+'%2F'+str(j).zfill(2)+'%2F'+str(i)
                    urls.append(url)
            else:
                # Para el resto recorremos 31 días
                for k in range(31,0,-1):
                    url = 'https://tarifaluzhora.es/?tarifa=pcb&fecha='+str(k).zfill(2)+'%2F'+str(j).zfill(2)+'%2F'+str(i)
                    urls.append(url)

In [182]:
# ELiminamos el 31 que es mañana y la fecha de hoy, que ya la tenemos
urls.pop(0)
urls.pop(0)

'https://tarifaluzhora.es/?tarifa=pcb&fecha=30%2F03%2F2022'

In [ ]:
# Recorremos el array de urls
for i in urls:
    # Fijamos url de la web
    url = i

    # Hacemos la petición a la página
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Obtenemos los precios
    precios = soup.find_all('span', itemprop="price")

    # Obtenemos la fecha
    fecha = soup.find('input', {'name': 'date'}).get('value')
    
    # Creamos un array con el contenido de las horas
    columnas = ['fecha']
    for h in horas:
        columnas.append(h.text)

    # Creamos un array con el contenido de los precios
    contenido = [fecha]
    for p in precios:
        contenido.append(p.text)
        
    # Creamos el df
    df1 = pd.DataFrame(data=[np.array(contenido)], columns=columnas)
    
    # Lo unimos al df original para crear la bd
    df = pd.concat([df,df1])

In [ ]:
# Cabecera del dataset
df.head()

In [ ]:
# Cola del dataset
df.tail()

A partir de aquí podemos añadir columnas de media y desviación. 

Sacar mínimos, máximos y tendencias.

Hacer gráficas y predicciones.

...